In [1]:
# Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from category_encoders.target_encoder import TargetEncoder

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from sklearn.utils.class_weight import compute_class_weight

## Cleaning the Event Dataset

This dataset contains records of everything that happened during a match, whether a player scored, getting booked or getting injured.

In [2]:
# Importing the data
df_events = pd.read_csv("match_event.csv")

# Sampling
df_events.sample(5)

,Match,Game,Minute,Home Team,Away Team,Home Player,Home Action,Home Score,Away Player,Away Action,Away Score,Home Event,Away Event
438,8,6,57,SC Heerenveen,Ajax,7. O. Dembélé,Assist: Soler,1-0,NaN,NaN,NaN,icon-matchevent-goal,NaN
1301,22,8,17,Heracles Almelo,FC Groningen,NaN,NaN,NaN,7. Foden,Assist: Maddison,0-1,NaN,icon-matchevent-goal
368,7,5,66,FC Groningen,Almere City,NaN,NaN,NaN,24. Mattoir,Erasing the referee's spray,NaN,NaN,icon-player-yellowcard
516,10,1,16,Almere City,AZ,7. Guillaume,Assist: Nalic,1-0,NaN,NaN,NaN,icon-matchevent-goal,NaN
109,3,1,80,AZ,PSV,NaN,NaN,NaN,1. Benítez,Failing to keep the required distance from a f...,NaN,NaN,icon-player-yellowcard


In [3]:
# Checking the information of the event data
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2329 entries, 0 to 2328
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Match        2329 non-null   int64 
 1   Game         2329 non-null   int64 
 2   Minute       2329 non-null   int64 
 3   Home Team    2329 non-null   object
 4   Away Team    2329 non-null   object
 5   Home Player  1220 non-null   object
 6   Home Action  1106 non-null   object
 7   Home Score   582 non-null    object
 8   Away Player  1106 non-null   object
 9   Away Action  1001 non-null   object
 10  Away Score   484 non-null    object
 11  Home Event   1222 non-null   object
 12  Away Event   1107 non-null   object
dtypes: int64(3), object(10)
memory usage: 236.7+ KB


In [4]:
# Creating new columns to store all informations together
df_events['Player'] = df_events['Home Player'].fillna("") + df_events['Away Player'].fillna("")
df_events['Action'] = df_events['Home Action'].fillna("") + df_events['Away Action'].fillna("")
df_events['Event'] = (df_events['Home Event'].fillna("").apply(lambda x: f"Home: {x}" if x else "") + 
                      df_events['Away Event'].fillna("").apply(lambda x: f"Away: {x}" if x else ""))
df_events.sample(3)

,Match,Game,Minute,Home Team,Away Team,Home Player,Home Action,Home Score,Away Player,Away Action,Away Score,Home Event,Away Event,Player,Action,Event
904,16,3,78,NEC Nijmegen,Willem II,NaN,NaN,NaN,35. Razak,Neymar,NaN,NaN,icon-matchevent-sub,35. Razak,Neymar,Away: icon-matchevent-sub
1635,28,9,27,Heracles Almelo,Willem II,10. Wirtz,Making multiple deliberate fouls,NaN,NaN,NaN,NaN,icon-player-yellowcard,NaN,10. Wirtz,Making multiple deliberate fouls,Home: icon-player-yellowcard
2119,35,8,42,NAC Breda,Fortuna Sittard,NaN,NaN,NaN,7. Hwang Hee-chan,Assist: Provod,0-2,NaN,icon-matchevent-goal,7. Hwang Hee-chan,Assist: Provod,Away: icon-matchevent-goal


In [5]:
# Choosing the appropriate columns
df_events.loc[:,['Match', 'Game', 'Home Team', 'Away Team', 'Minute', 'Player', 'Action', 'Event']].sample(10)

,Match,Game,Home Team,Away Team,Minute,Player,Action,Event
1106,19,4,FC Utrecht,SC Heerenveen,21,12. Vitinha,Assist: Havertz,Home: icon-matchevent-goal
862,15,5,NAC Breda,Almere City,36,7. Garbett,Attempting to kick the ball in a dangerous manner,Home: icon-player-yellowcard
1167,20,7,FC Utrecht,AZ,79,3. Bremer,Denying an obvious goal-scoring opportunity,Home: icon-player-yellowcard
2088,35,3,Willem II,NEC Nijmegen,81,9. Benzema,15. Messi,Home: icon-matchevent-sub
856,15,4,SC Heerenveen,FC Twente,81,2. Alexander-Arnold,Making multiple deliberate fouls,Home: icon-player-yellowcard
444,8,7,Go Ahead Eagles,Fortuna Sittard,23,11. Leão,Assist: J. Kramer,Home: icon-matchevent-goal
901,16,2,AZ,Heracles Almelo,80,11. J. Clarke,7. van Bommel,Home: icon-matchevent-sub
616,11,7,FC Utrecht,Go Ahead Eagles,27,Faes,6. Bastoni,Home: icon-matchevent-sub
2062,34,9,NEC Nijmegen,Heracles Almelo,73,16. Szoboszlai,Assist: Freuler,Home: icon-matchevent-goal
1088,19,2,Ajax,Go Ahead Eagles,90,24. van de Ven,Kicking the ball away after the referee's whistle,Home: icon-player-yellowcard


In [6]:
# Checking the numbers of each levels within the event column
df_events['Event'].value_counts()

Event
Home: icon-matchevent-goal                   557
Away: icon-matchevent-goal                   465
Away: icon-player-yellowcard                 313
Home: icon-player-yellowcard                 294
Home: icon-matchevent-sub                    266
Away: icon-matchevent-sub                    257
Home: icon-player-injury                      56
Away: icon-player-injury                      38
Home: icon-matchevent-penaltygoal             25
Away: icon-matchevent-penaltygoal             19
Home: icon-player-redcard                     16
Away: icon-player-redcard                     11
Home: icon-matchevent-penaltymiss              6
Away: icon-matchevent-redcardsecondyellow      3
Home: icon-matchevent-redcardsecondyellow      2
Away: icon-matchevent-penaltymiss              1
Name: count, dtype: int64

In [8]:
# Splitting the column
df_events[['Location', 'EventType']] = df_events['Event'].str.split(':', n=1, expand=True)

# Cleaning up the location column
df_events['Location'] = df_events['Location'].str.strip()

# Cleaning up the Event Type column
# Function to extract the event type after the second dash
def extract_event_type(full_event):
    if pd.isna(full_event):
        return pd.NA
    parts = full_event.split('-')
    if len(parts) >= 3:
        return parts[-1].strip()
    return full_event.strip()

# Apply the function to create the 'Event_Type' column
df_events['EventType'] = df_events['EventType'].apply(extract_event_type)

df_events.loc[:,['Location', 'EventType']].sample(10)

,Location,EventType
641,Home,goal
1788,Away,sub
1173,Home,yellowcard
258,Away,sub
1594,Home,sub
614,Away,sub
650,Home,goal
1276,Away,yellowcard
2051,Away,sub
355,Home,yellowcard


In [9]:
# Fixing the team column
df_events['Team'] = np.where(df_events['Location'] == "Home", df_events['Home Team'], df_events['Away Team'])
df_events['Opponent'] = np.where(df_events['Location'] == "Home", df_events['Away Team'], df_events['Home Team'])

In [10]:
# Function to clean player names
def clean_player_name(name):
    if pd.isna(name):
        return name
    # This regex matches one or two numbers at the start, followed by a period and optional space
    return pd.Series(name).str.replace(r'^\d{1,2}\.\s*', '', regex=True)

# Apply the function to the player column(s)
df_events['Player'] = df_events['Player'].apply(clean_player_name)
df_events['Action'] = df_events['Action'].apply(clean_player_name)

In [11]:
def remove_word(value):
    if pd.isna(value):
        return value
    
    # Convert to string if it's not already
    value = str(value)
    
    return value.replace('Assist:', '').strip()

# Apply the function to the 'Action' column
# df_events['Action'] = df_events['Action'].apply(remove_word)

In [12]:
# Final dataset
df_finalEvents = df_events.loc[:,['Match', 'Game', 'Team', 'Opponent', 'Minute', 'Player', 'Action', 'Location', 'EventType']]

df_finalEvents.tail(10)

,Match,Game,Team,Opponent,Minute,Player,Action,Location,EventType
2319,39,8,Willem II,FC Twente,62,Messi,,Away,goal
2320,39,8,FC Twente,Willem II,77,Veltman,Guerreiro,Home,sub
2321,39,9,Fortuna Sittard,Heracles Almelo,40,T. Martínez,Sprained leg muscle,Home,injury
2322,39,9,Fortuna Sittard,Heracles Almelo,40,Barnes,T. Martínez,Home,sub
2323,39,9,Heracles Almelo,Fortuna Sittard,42,Núñez,Assist: Thuram,Away,goal
2324,39,9,Heracles Almelo,Fortuna Sittard,56,Wirtz,Attempting to kick the ball in a dangerous manner,Away,yellowcard
2325,39,9,Fortuna Sittard,Heracles Almelo,66,Rongier,Assist: Otávio,Home,goal
2326,39,9,Heracles Almelo,Fortuna Sittard,76,Yüksek,T. Bruns,Away,sub
2327,39,9,Fortuna Sittard,Heracles Almelo,87,Otávio,Assist: Aebischer,Home,goal
2328,39,9,Heracles Almelo,Fortuna Sittard,91,van Oorschot,Assist: Thuram,Away,goal


In [13]:
# Exporting the dataset
df_finalEvents.to_csv("events_cleaned.csv", index=False)

## Preparing the player rating dataset

#### This data contains the performance of each player from each match. There are missing data on the 39th match. The goal here is to clean the data, integrate external data to it and use that to build a deep learning model to populate the missing values.

In [14]:
# Importing the player rating dataset
df_rating = pd.read_csv("Cleaned_dataset//player_ratings.csv")
df_rating = df_rating.drop(['Unnamed: 6', 'Unnamed: 7'], axis=1)

In [15]:
# Checking the data types
df_finalEvents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2329 entries, 0 to 2328
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Match      2329 non-null   int64 
 1   Game       2329 non-null   int64 
 2   Team       2329 non-null   object
 3   Opponent   2329 non-null   object
 4   Minute     2329 non-null   int64 
 5   Player     2329 non-null   object
 6   Action     2329 non-null   object
 7   Location   2329 non-null   object
 8   EventType  2329 non-null   object
dtypes: int64(3), object(6)
memory usage: 163.9+ KB


In [16]:
# Checking the structure of the player rating data
df_rating.sample(3)

,Match,Game,Team,Home_or_Away,Name,Rating
7333,37,8,SC Heerenveen,Home,Gyökeres,9
6879,35,7,Go Ahead Eagles,Away,Alexander-Arnold,-
4002,21,5,AZ,Home,Wolfe,-


#### In this section, I will restructure the events dataset, so I can join that with the player rating dataset.

In [17]:
# Checking the levels in the event column
df_finalEvents['EventType'].unique()

array(['goal', 'yellowcard', 'penaltymiss', 'sub', 'penaltygoal',
       'injury', 'redcard', 'redcardsecondyellow'], dtype=object)

In [18]:
# Create a dataframe for assists
df_assists = df_finalEvents[df_finalEvents['Action'].str.contains('Assist:', na=False)].copy()
# Clean the 'Action' column in the assist table
df_assists['Action'] = df_assists['Action'].apply(lambda x: x.replace('Assist: ', '').strip() if isinstance(x, str) else x)
player_assists = df_assists.groupby(['Team', 'Match', 'Game', 'Action']).size().reset_index(name='Assist')

In [19]:
# Group by and aggregate
summaryEvent = df_finalEvents.groupby(['Match', 'Game', 'Team', 'Player', 'EventType']).size().unstack(fill_value=0)

In [20]:
# Group by and aggregate
summaryEvent = df_finalEvents.groupby(['Match', 'Game', 'Team', 'Player', 'EventType']).size().unstack(fill_value=0) # unstack pivots the column

# Reset the index
summaryEvent = summaryEvent.reset_index()

# Ensure all category columns exist, add if missing
for category in ['goal', 'yellowcard', 'redcard', 'redcardsecondyellow', 'injury', 'penaltygoal', 'penaltymiss', 'sub']:
    if category not in summaryEvent.columns:
        summaryEvent[category] = 0

# Create a dictionary of opponents for each match and game
opponent_dict = df_finalEvents.groupby(['Match', 'Game', 'Team'])['Opponent'].first().to_dict()

# Add the 'Opponent' column
def get_opponent(row):
    key = (row['Match'], row['Game'], row['Team'])
    return opponent_dict.get(key, 'Unknown')

summaryEvent['Opponent'] = summaryEvent.apply(get_opponent, axis=1)

# Reorder columns
column_order = ['Match', 'Game', 'Team', 'Opponent', 'Player', 'goal', 'yellowcard', 'redcard', 'redcardsecondyellow', 
                'injury', 'penaltygoal', 'penaltymiss', 'sub']
summaryEvent = summaryEvent[column_order]

# Sort the DataFrame
summaryEvent = summaryEvent.sort_values(['Match', 'Game', 'Team', 'Player'])


In [21]:
# Making sure that penalties are considered as goals, while misses aren't
summaryEvent['goal'] = np.where(summaryEvent['penaltygoal'] != 0, 
                           summaryEvent['penaltygoal'] + summaryEvent['goal'], 
                           summaryEvent['goal'])

In [22]:
# Checking the data
summaryEvent[summaryEvent['penaltygoal'] != 0].sample(3)

EventType,Match,Game,Team,Opponent,Player,goal,yellowcard,redcard,redcardsecondyellow,injury,penaltygoal,penaltymiss,sub
260,6,8,Willem II,FC Utrecht,Lukaku,1,0,0,0,0,1,0,0
51,2,1,FC Utrecht,Almere City,Gregoritsch,1,0,0,0,0,1,0,0
1874,36,8,Go Ahead Eagles,Willem II,Openda,1,0,0,0,0,1,0,0


In [23]:
# Merging the player rating dataset with the event summary dataset
summaryEvent = pd.merge(df_rating, summaryEvent, 
                   left_on=['Match', 'Game', 'Team', 'Name'],
                   right_on=['Match', 'Game', 'Team', 'Player'],
                   how='left')

In [24]:
# List of columns to convert
columns_to_convert = ['goal', 'yellowcard', 'redcard', 'redcardsecondyellow', 
                      'injury', 'penaltygoal', 'penaltymiss', 'sub']

# Convert each column to nullable integer
for column in columns_to_convert:
    summaryEvent[column] = summaryEvent[column].astype('Int64')
    summaryEvent[column] = summaryEvent[column].fillna(0)

In [25]:
# Removing the duplicated player column
summaryEvent.drop(['Player'], axis=1, inplace=True)

#### This section focuses on creating a dataset that contains the results of each match including adding the opponent's results.

In [26]:
summaryOutcome = summaryEvent.loc[:, ['Match', 'Game', 'Team', 'goal']].copy()

# Summarising the outcome
summaryOutcome = summaryOutcome.groupby(['Match', 'Game', 'Team']).sum('goal').reset_index()

In [27]:
# Importing fixtures
fixture_df = pd.read_csv("D:\Webscraping\Eridivisie\Cleaned_dataset\match_fixtures.csv")

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_27084\184536023.py:2: SyntaxWarning: invalid escape sequence '\W'
  fixture_df = pd.read_csv("D:\Webscraping\Eridivisie\Cleaned_dataset\match_fixtures.csv")


In [28]:
# Merging the main dataset with the fixtures to create an opponent column
summary_fixtures = pd.merge(fixture_df, summaryOutcome,
                            left_on=['match', 'HomeTeam'],
                            right_on=['Match', 'Team'],
                            how='left')

summary_fixtures = summary_fixtures.loc[:, ['match', 'HomeTeam', 'AwayTeam', 'Game', 'goal']]

summary_fixtures = summary_fixtures.rename(columns={
    'HomeTeam': 'Team',
    'AwayTeam': 'Opponent'
})

In [29]:
# First, let's create a copy of the DataFrame to work with
df_fixture = summary_fixtures.copy()

# Identify rows with NA values
na_rows = df_fixture[df_fixture['Game'].isna() | df_fixture['goal'].isna()]

for index, row in na_rows.iterrows():
    # Find the corresponding row where HomeTeam and AwayTeam are swapped
    corresponding_row = df_fixture[(df_fixture['match'] == row['match']) & 
                           (df_fixture['Team'] == row['Opponent']) & 
                           (df_fixture['Opponent'] == row['Team'])]
    
    if not corresponding_row.empty:
        # Fill the NA values with the information from the corresponding row
        df_fixture.loc[index, 'Game'] = corresponding_row['Game'].values[0]
        df_fixture.loc[index, 'goal'] = corresponding_row['goal'].values[0]

In [30]:
df_fixture[df_fixture['match'] == 1]

,match,Team,Opponent,Game,goal
0,1,Ajax,FC Groningen,1,2
1,1,NEC Nijmegen,PEC Zwolle,2,3
2,1,Sparta Rotterdam,Feyenoord,3,0
3,1,NAC Breda,RKC Waalwijk,4,2
4,1,Go Ahead Eagles,AZ,5,0
5,1,Fortuna Sittard,PSV,6,0
6,1,FC Utrecht,SC Heerenveen,7,2
7,1,Willem II,Almere City,8,1
8,1,Heracles Almelo,FC Twente,9,3
323,1,FC Groningen,Ajax,1,1


In [31]:
df_fixture.sort_values(['match', 'Team'], ascending=True).reset_index(inplace=True)

In [32]:
summary_duplicate = df_fixture.copy()

In [33]:
# Merging the duplicate with the same dataset
df_fixture = pd.merge(df_fixture, summary_duplicate,
                            left_on=['match', 'Game', 'Team'],
                            right_on=['match', 'Game', 'Opponent'],
                            how='left')

In [34]:
df_fixture.head()

,match,Team_x,Opponent_x,Game,goal_x,Team_y,Opponent_y,goal_y
0,1,Ajax,FC Groningen,1,2,FC Groningen,Ajax,1
1,1,NEC Nijmegen,PEC Zwolle,2,3,PEC Zwolle,NEC Nijmegen,0
2,1,Sparta Rotterdam,Feyenoord,3,0,Feyenoord,Sparta Rotterdam,1
3,1,NAC Breda,RKC Waalwijk,4,2,RKC Waalwijk,NAC Breda,0
4,1,Go Ahead Eagles,AZ,5,0,AZ,Go Ahead Eagles,0


In [36]:
# Creating a new column to determine the outcome of a team from each match
df_fixture['Outcome'] = np.where(df_fixture['goal_x'] > df_fixture['goal_y'], 'W',
                                     np.where(df_fixture['goal_x'] < df_fixture['goal_y'], 'L', 'D'))

In [37]:
# Merging the player ratings with the fixture table
rating_merged = pd.merge(summaryEvent, df_fixture,
                         left_on=['Match', 'Game', 'Team'],
                         right_on=['match', 'Game', 'Team_x'],
                         how='left')

In [38]:
# Choosing the appropriate columns
rating_merged = rating_merged.loc[:, ['Match', 'Game', 'Team', 'Opponent_x', 'Home_or_Away', 'Name',
                                      'Rating', 'goal', 'yellowcard', 'redcard', 'redcardsecondyellow',
                                      'injury', 'penaltygoal', 'penaltymiss', 'sub', 'Outcome']]

#### This section focuses on enriching the player rating dataset with more data such as the player of the match and the results of each match as a team.

In [39]:
# Importing the match summary dataset and the man of the match dataset
player_match_df = pd.read_csv("player_referee.csv")
scoresheet = pd.read_csv("Cleaned_dataset//scoresheet.csv")

In [40]:
# Apply the function to the player of the match column
player_match_df['Player of the Match'] = player_match_df['Player of the Match'].apply(clean_player_name)

In [41]:
# Merging the main dataset with the player of the match dataset
rating_merged_df = pd.merge(rating_merged, player_match_df,
                            left_on=['Match', 'Game', 'Name'],
                            right_on=['Match', 'Game', 'Player of the Match'],
                            how='left').drop(['Referee', 'Mood'], axis=1)

In [42]:
# Changing the player of the match to binary
rating_merged_df['Player of the Match'] = np.where(rating_merged_df['Player of the Match'].isna(), 0, 1)

In [43]:
# Merging the main dataset with the scoresheet
ratingDF = pd.merge(rating_merged_df, scoresheet,
                    left_on=['Match', 'Team', 'Opponent_x'],
                    right_on=['match', 'Team', 'Opponent'],
                    how='left').drop(['match', 'Opponent'], axis=1)

#### This section focuses on adding the assist variable.

In [46]:
# Merging the overall dataset with the assist dataset
player_assists.head()

,Team,Match,Game,Action,Assist
0,AZ,3,1,Mijnans,1
1,AZ,3,1,Penetra,1
2,AZ,5,7,Goudmijn,1
3,AZ,5,7,J. Clarke,1
4,AZ,6,1,Goudmijn,1


In [47]:
ratingDF = pd.merge(ratingDF, player_assists,
                         left_on=['Match', 'Game', 'Team', 'Name'],
                         right_on=['Match', 'Game', 'Team', 'Action'],
                         how='left')

In [48]:
ratingDF.drop('Action', axis=1, inplace=True)

In [49]:
ratingDF['Assist'] = ratingDF['Assist'].fillna(0)

In [366]:
# ratingDF.to_csv("D:\Webscraping\Eridivisie\Cleaned_dataset\Deep Learning\player_summary.csv", index=True)

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_16888\3576469562.py:1: SyntaxWarning: invalid escape sequence '\W'
  ratingDF.to_csv("D:\Webscraping\Eridivisie\Cleaned_dataset\Deep Learning\player_summary.csv", index=True)


#### Adding the player's value and their positions to the player rating dataset

In [50]:
# Importing the transfers dataset
transfers_df = pd.read_csv("player_transfers.csv")

In [52]:
transfers_df.head()

,Player,From,To,Position,Week,Value,Price,Date
0,Alaba,Real Madrid,Go Ahead Eagles,CB,39,22.3M,31.9M,06:11
1,Leno,Liverpool,Go Ahead Eagles,GK,39,21.2M,26.7M,06:11
2,White,Real Madrid,Go Ahead Eagles,RB,39,15.6M,21.9M,06:11
3,Mitoma,Barcelona,Go Ahead Eagles,LW,39,22.8M,28.3M,06:11
4,Guirassy,Go Ahead Eagles,Arsenal,ST,38,12.3M,24.7M,4 September


In [53]:
# Creating a list for Eridivisie teams
eridivisie_teams = ratingDF['Team'].unique()

In [54]:
# Obtaining the maximum value of a player
filtered_transfers = (
    transfers_df.groupby(['Player', 'To', 'Position'])['Value']
    .max()
    .reset_index()
    .loc[lambda df: df['To'].isin(eridivisie_teams)]
)

In [57]:
# Checking the first few observations
filtered_transfers.head()

,Player,To,Position,Value
2,A. Andresen,Fortuna Sittard,LM,1.3M
3,A. Jovanovic,Fortuna Sittard,GK,2.0M
5,A. Muñoz,FC Utrecht,LB,4.7M
7,A. Pérez,Go Ahead Eagles,LW,7.4M
11,A. Varela,Go Ahead Eagles,CDM,7.5M


In [58]:
# Numbers of transfers from each club
filtered_transfers['To'].value_counts()

To
Go Ahead Eagles     127
Willem II            96
Ajax                 70
FC Groningen         56
SC Heerenveen        51
FC Utrecht           45
Fortuna Sittard      41
FC Twente            31
NEC Nijmegen         25
Heracles Almelo      21
Feyenoord            18
NAC Breda            14
AZ                   12
Sparta Rotterdam      8
PSV                   5
RKC Waalwijk          3
PEC Zwolle            2
Name: count, dtype: int64

#### Squad dataset contains the most recent players from each team. I will add this to the player rating dataset then the transfers dataset

In [59]:
# Importing the squad dataset
squad_df = pd.read_csv("player_team.csv")
squad_df.head()

,Team,Player,Position,Nationality,Age,Value
0,Go Ahead Eagles,Openda,ST,Belgian,24,155M
1,Go Ahead Eagles,Zirkzee,ST,Dutch,23,128M
2,Go Ahead Eagles,Lamine Yamal,RW,Spanish,17,116M
3,Go Ahead Eagles,Kulusevski,RW,Swedish,24,113M
4,Go Ahead Eagles,Leão,LW,Portuguese,25,112M


In [60]:
# Number of players from each club
squad_df['Team'].value_counts()

Team
Go Ahead Eagles     36
Ajax                25
Willem II           21
PEC Zwolle          21
NEC Nijmegen        19
FC Utrecht          19
Feyenoord           19
Almere City         18
AZ                  18
SC Heerenveen       18
PSV                 17
FC Twente           17
NAC Breda           17
FC Groningen        16
Heracles Almelo     16
Fortuna Sittard     15
Sparta Rotterdam    14
RKC Waalwijk        13
Name: count, dtype: int64

In [61]:
ratingDF.head()

,Match,Game,Team,Opponent_x,Home_or_Away,Name,Rating,goal,yellowcard,redcard,redcardsecondyellow,injury,penaltygoal,penaltymiss,sub,Outcome,Player of the Match,TeamGoals,Conceded,Assist
0,1,1,FC Groningen,Ajax,Away,Verbruggen,5,0,0,0,0,0,0,0,0,L,0,1,2,0.0
1,1,1,FC Groningen,Ajax,Away,Benedetti,7,0,0,0,0,0,0,0,0,L,0,1,2,1.0
2,1,1,FC Groningen,Ajax,Away,Gabriel Paulista,6,0,0,0,0,0,0,0,0,L,0,1,2,0.0
3,1,1,FC Groningen,Ajax,Away,Thiaw,6,0,0,0,0,0,0,0,0,L,0,1,2,0.0
4,1,1,FC Groningen,Ajax,Away,Dedic,5,0,1,0,0,0,0,0,0,L,0,1,2,0.0


In [93]:
# Merging the main dataset with the squad dataset
rating_update = pd.merge(ratingDF, squad_df,
                         left_on=['Team', 'Name'],
                         right_on=['Team', 'Player'],
                         how='left').drop(['Nationality', 'Age'], axis=1)

In [94]:
# Number of missing values from each column
rating_update.isna().sum()

Match                     0
Game                      0
Team                      0
Opponent_x                0
Home_or_Away              0
Name                      0
Rating                  201
goal                      0
yellowcard                0
redcard                   0
redcardsecondyellow       0
injury                    0
penaltygoal               0
penaltymiss               0
sub                       0
Outcome                   0
Player of the Match       0
TeamGoals                 0
Conceded                  0
Assist                    0
Player                 1350
Position               1350
Value                  1350
dtype: int64

In [95]:
# Create dictionaries for each column you want to fill
player_dict = dict(zip(transfers_df['Player'], transfers_df['Player']))
value_dict = dict(zip(transfers_df['Player'], transfers_df['Value']))
position_dict = dict(zip(transfers_df['Player'], transfers_df['Position']))

In [96]:
# Fill NA values only
rating_update['Player'] = rating_update['Player'].fillna(rating_update['Name'].map(player_dict))
rating_update['Value'] = rating_update['Value'].fillna(rating_update['Name'].map(value_dict))
rating_update['Position'] = rating_update['Position'].fillna(rating_update['Name'].map(position_dict))

In [97]:
# Checking for missing values again
rating_update.isna().sum()

Match                    0
Game                     0
Team                     0
Opponent_x               0
Home_or_Away             0
Name                     0
Rating                 201
goal                     0
yellowcard               0
redcard                  0
redcardsecondyellow      0
injury                   0
penaltygoal              0
penaltymiss              0
sub                      0
Outcome                  0
Player of the Match      0
TeamGoals                0
Conceded                 0
Assist                   0
Player                   0
Position                 0
Value                    0
dtype: int64

In [98]:
# Removing the duplicated column
rating_update = rating_update.drop('Player', axis=1)

In [99]:
# Checking the structure of the dataset
rating_update.head()

,Match,Game,Team,Opponent_x,Home_or_Away,Name,Rating,goal,yellowcard,redcard,...,penaltygoal,penaltymiss,sub,Outcome,Player of the Match,TeamGoals,Conceded,Assist,Position,Value
0,1,1,FC Groningen,Ajax,Away,Verbruggen,5,0,0,0,...,0,0,0,L,0,1,2,0.0,GK,3.8M
1,1,1,FC Groningen,Ajax,Away,Benedetti,7,0,0,0,...,0,0,0,L,0,1,2,1.0,LB,3.2M
2,1,1,FC Groningen,Ajax,Away,Gabriel Paulista,6,0,0,0,...,0,0,0,L,0,1,2,0.0,CB,3.0M
3,1,1,FC Groningen,Ajax,Away,Thiaw,6,0,0,0,...,0,0,0,L,0,1,2,0.0,CB,4.8M
4,1,1,FC Groningen,Ajax,Away,Dedic,5,0,1,0,...,0,0,0,L,0,1,2,0.0,RB,4.7M


In [100]:
# Cleaning the value column by removing the letters and turning them into millions or thousands
def convert_value(value):
    if pd.isna(value):
        return np.nan
    value = str(value).strip()
    if value.endswith('M'):
        return float(value[:-1]) * 1_000_000
    elif value.endswith('K'):
        return float(value[:-1]) * 1_000
    else:
        try:
            return float(value)
        except ValueError:
            return np.nan
        
# Convert the 'Value' column in transfers_df
rating_update['Value'] = rating_update['Value'].apply(convert_value)

In [101]:
# Converting the assist column to integer
rating_update['Assist'] = rating_update['Assist'].astype(int)

In [102]:
# Checking the data type
rating_update.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7591 entries, 0 to 7590
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Match                7591 non-null   int64  
 1   Game                 7591 non-null   int64  
 2   Team                 7591 non-null   object 
 3   Opponent_x           7591 non-null   object 
 4   Home_or_Away         7591 non-null   object 
 5   Name                 7591 non-null   object 
 6   Rating               7390 non-null   object 
 7   goal                 7591 non-null   Int64  
 8   yellowcard           7591 non-null   Int64  
 9   redcard              7591 non-null   Int64  
 10  redcardsecondyellow  7591 non-null   Int64  
 11  injury               7591 non-null   Int64  
 12  penaltygoal          7591 non-null   Int64  
 13  penaltymiss          7591 non-null   Int64  
 14  sub                  7591 non-null   Int64  
 15  Outcome              7591 non-null   o

In [103]:
# First few observations
rating_update.head()

,Match,Game,Team,Opponent_x,Home_or_Away,Name,Rating,goal,yellowcard,redcard,...,penaltygoal,penaltymiss,sub,Outcome,Player of the Match,TeamGoals,Conceded,Assist,Position,Value
0,1,1,FC Groningen,Ajax,Away,Verbruggen,5,0,0,0,...,0,0,0,L,0,1,2,0,GK,3800000.0
1,1,1,FC Groningen,Ajax,Away,Benedetti,7,0,0,0,...,0,0,0,L,0,1,2,1,LB,3200000.0
2,1,1,FC Groningen,Ajax,Away,Gabriel Paulista,6,0,0,0,...,0,0,0,L,0,1,2,0,CB,3000000.0
3,1,1,FC Groningen,Ajax,Away,Thiaw,6,0,0,0,...,0,0,0,L,0,1,2,0,CB,4800000.0
4,1,1,FC Groningen,Ajax,Away,Dedic,5,0,1,0,...,0,0,0,L,0,1,2,0,RB,4700000.0


In [105]:
summaryEvent

,Match,Game,Team,Home_or_Away,Name,Rating,Opponent,goal,yellowcard,redcard,redcardsecondyellow,injury,penaltygoal,penaltymiss,sub
0,1,1,FC Groningen,Away,Verbruggen,5,NaN,0,0,0,0,0,0,0,0
1,1,1,FC Groningen,Away,Benedetti,7,NaN,0,0,0,0,0,0,0,0
2,1,1,FC Groningen,Away,Gabriel Paulista,6,NaN,0,0,0,0,0,0,0,0
3,1,1,FC Groningen,Away,Thiaw,6,NaN,0,0,0,0,0,0,0,0
4,1,1,FC Groningen,Away,Dedic,5,Ajax,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7586,39,9,Heracles Almelo,Away,Núñez,NaN,Fortuna Sittard,1,0,0,0,0,0,0,0
7587,39,9,Heracles Almelo,Away,Batshuayi,NaN,NaN,0,0,0,0,0,0,0,0
7588,39,9,Heracles Almelo,Away,van Oorschot,NaN,Fortuna Sittard,1,0,0,0,0,0,0,0
7589,39,9,Heracles Almelo,Away,Thuram,NaN,NaN,0,0,0,0,0,0,0,0


In [104]:
# Exporting the dataset
rating_update.to_csv("D:\Webscraping\Eridivisie\Cleaned_dataset\Ready\playerRating.csv", index=False)

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_27084\1185224141.py:2: SyntaxWarning: invalid escape sequence '\W'
  rating_update.to_csv("D:\Webscraping\Eridivisie\Cleaned_dataset\Ready\playerRating.csv", index=False)


#### This is the match summary dataset and this contains information of the team performance, such as the ball possession, corners taken, shots made, etc. I will add this to the player rating dataset.

In [73]:
# Importing match summary dataset
matchSummary = pd.read_csv("D:\Webscraping\Eridivisie\Cleaned_dataset\match_summary_cleaned.csv")
matchSummary.head()

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_27084\2972623300.py:2: SyntaxWarning: invalid escape sequence '\W'
  matchSummary = pd.read_csv("D:\Webscraping\Eridivisie\Cleaned_dataset\match_summary_cleaned.csv")


,match,Team,Opponent,Location,Goals,Conceded,ShotsTaken,ShotsConceded,Corners,CornersConceded,Foul,Yellow,Red,Possession,OpponentPossession,TeamFormation,OpponentFormation,TeamField,CentreField,OpponentField
0,1,Ajax,FC Groningen,Home,2,1,14,11,5,5,15,3,0,57,43,4-3-3 B,4-3-3 A,23,49,28
1,1,NEC Nijmegen,PEC Zwolle,Home,3,0,16,6,8,4,13,2,0,63,37,4-3-3 A,4-3-3 B,15,53,32
2,1,Sparta Rotterdam,Feyenoord,Home,0,1,9,12,3,3,10,0,0,41,59,4-3-3 A,4-4-2 B,25,55,20
3,1,NAC Breda,RKC Waalwijk,Home,2,0,15,5,10,4,11,1,0,62,38,4-4-2 B,4-3-3 B,16,55,29
4,1,Go Ahead Eagles,AZ,Home,0,0,13,7,4,1,16,2,0,60,40,4-3-3 B,4-3-3 B,18,56,26


In [74]:
# Choosing the necessary columns
matchSummary = matchSummary.loc[:, ['match', 'Team', 'Opponent', 'ShotsTaken', 'ShotsConceded', 'Corners',
                                    'CornersConceded', 'Foul', 'Possession', 'TeamFormation', 'OpponentFormation',
                                    'TeamField', 'OpponentField']]

In [75]:
# Seeing the common columns for merging
rating_update.columns, matchSummary.columns

(Index(['Match', 'Game', 'Team', 'Opponent_x', 'Home_or_Away', 'Name', 'Rating',
        'goal', 'yellowcard', 'redcard', 'redcardsecondyellow', 'injury',
        'penaltygoal', 'penaltymiss', 'sub', 'Outcome', 'Player of the Match',
        'TeamGoals', 'Conceded', 'Assist', 'Position', 'Value'],
       dtype='object'),
 Index(['match', 'Team', 'Opponent', 'ShotsTaken', 'ShotsConceded', 'Corners',
        'CornersConceded', 'Foul', 'Possession', 'TeamFormation',
        'OpponentFormation', 'TeamField', 'OpponentField'],
       dtype='object'))

In [76]:
# Merging the datasets
rating_update = pd.merge(rating_update, matchSummary,
                         left_on=['Match', 'Team', 'Opponent_x'],
                         right_on=['match', 'Team', 'Opponent'],
                         how='left')

In [77]:
# Removing the duplicated columns
rating_update = rating_update.drop(['match', 'Opponent'], axis=1)

In [78]:
# Checking the datatypes
rating_update.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7591 entries, 0 to 7590
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Match                7591 non-null   int64  
 1   Game                 7591 non-null   int64  
 2   Team                 7591 non-null   object 
 3   Opponent_x           7591 non-null   object 
 4   Home_or_Away         7591 non-null   object 
 5   Name                 7591 non-null   object 
 6   Rating               7390 non-null   object 
 7   goal                 7591 non-null   Int64  
 8   yellowcard           7591 non-null   Int64  
 9   redcard              7591 non-null   Int64  
 10  redcardsecondyellow  7591 non-null   Int64  
 11  injury               7591 non-null   Int64  
 12  penaltygoal          7591 non-null   Int64  
 13  penaltymiss          7591 non-null   Int64  
 14  sub                  7591 non-null   Int64  
 15  Outcome              7591 non-null   o

## Preparing data for machine learning

#### Separating the players with ratings from players without ratings. These players are marked with a dash because they only played for a few minutes in the game.

In [79]:
# Removing rows with '-' in the rating column
unrated_players_df = rating_update[rating_merged['Rating'] == '-']

In [81]:
# Checking the dataset
unrated_players_df.head()

,Match,Game,Team,Opponent_x,Home_or_Away,Name,Rating,goal,yellowcard,redcard,...,ShotsTaken,ShotsConceded,Corners,CornersConceded,Foul,Possession,TeamFormation,OpponentFormation,TeamField,OpponentField
44,1,2,NEC Nijmegen,PEC Zwolle,Home,Nuytinck,-,0,0,0,...,16,6,8,4,13,63,4-3-3 A,4-3-3 B,15,32
78,1,4,RKC Waalwijk,NAC Breda,Away,Baeza,-,0,0,0,...,5,15,4,10,14,38,4-3-3 B,4-4-2 B,29,16
112,1,5,Go Ahead Eagles,AZ,Home,Breum,-,0,0,0,...,13,7,4,1,16,60,4-3-3 B,4-3-3 B,18,26
124,1,6,PSV,Fortuna Sittard,Away,Dest,-,0,0,0,...,16,7,10,1,15,70,4-3-3 A,4-4-2 B,16,33
147,1,7,SC Heerenveen,FC Utrecht,Away,Kersten,-,0,0,0,...,8,13,6,5,13,40,4-3-3 B,4-3-3 A,26,20


In [309]:
# Exporting the unrated players
unrated_players_df.to_csv("unrated_players.csv", index=False)

#### Preparing the player rating dataset for encoding.

In [82]:
# Converting rating to integer
rating_update = rating_update[rating_update['Rating'] != '-']
rating_update['Rating'] = rating_update['Rating'].astype("Int64")

In [83]:
rating_update.columns

Index(['Match', 'Game', 'Team', 'Opponent_x', 'Home_or_Away', 'Name', 'Rating',
       'goal', 'yellowcard', 'redcard', 'redcardsecondyellow', 'injury',
       'penaltygoal', 'penaltymiss', 'sub', 'Outcome', 'Player of the Match',
       'TeamGoals', 'Conceded', 'Assist', 'Position', 'Value', 'ShotsTaken',
       'ShotsConceded', 'Corners', 'CornersConceded', 'Foul', 'Possession',
       'TeamFormation', 'OpponentFormation', 'TeamField', 'OpponentField'],
      dtype='object')

In [84]:
# Choosing variables
rating_update = rating_update.loc[:, ['Match', 'Game', 'Team', 'Opponent_x', 'Home_or_Away', 'Name', 'Rating', 'goal', 'Assist', 'yellowcard',
                              'redcard', 'redcardsecondyellow', 'injury', 'penaltygoal', 'penaltymiss', 'sub', 'Outcome',
                              'Player of the Match', 'TeamGoals', 'Conceded', 'Position', 'Value', 'ShotsTaken', 'ShotsConceded', 
                              'Corners', 'CornersConceded', 'Foul', 'Possession', 'TeamFormation', 'OpponentFormation', 'TeamField', 'OpponentField']]

In [314]:
# Checking the number of rows
len(rating_update)

7219

In [85]:
# Number of players from each position
rating_update['Position'].value_counts()

Position
CB     1392
CM     1113
ST      748
GK      680
LB      631
CAM     566
RW      566
CDM     514
RB      498
LW      464
LM       33
RM       14
Name: count, dtype: int64

In [86]:
# Number of unique values
rating_update.nunique()

Match                   39
Game                     9
Team                    18
Opponent_x              18
Home_or_Away             2
Name                   517
Rating                   9
goal                     5
Assist                   5
yellowcard               2
redcard                  2
redcardsecondyellow      2
injury                   2
penaltygoal              3
penaltymiss              2
sub                      2
Outcome                  3
Player of the Match      2
TeamGoals               10
Conceded                10
Position                12
Value                  282
ShotsTaken              25
ShotsConceded           25
Corners                 16
CornersConceded         16
Foul                    17
Possession              45
TeamFormation           13
OpponentFormation       13
TeamField               37
OpponentField           37
dtype: int64

## Encoding the variables

#### This section will be important for machine learning as I want to prevent ordering of levels.

In [87]:
# Encoding the categorical variables 
rating_update = pd.get_dummies(rating_update, prefix="location", columns=['Home_or_Away'], dtype=int)
rating_update = pd.get_dummies(rating_update, prefix="outcome", columns=['Outcome'], dtype=int)
rating_update = pd.get_dummies(rating_update, prefix="position", columns=['Position'], dtype=int)
rating_update = pd.get_dummies(rating_update, prefix="TeamForm", columns=['TeamFormation'], dtype=int)
rating_update = pd.get_dummies(rating_update, prefix="OppForm", columns=['OpponentFormation'], dtype=int)

rating_update.head()

,Match,Game,Team,Opponent_x,Name,Rating,goal,Assist,yellowcard,redcard,...,OppForm_3-4-3 B,OppForm_4-2-3-1,OppForm_4-3-3 A,OppForm_4-3-3 B,OppForm_4-4-2 A,OppForm_4-4-2 B,OppForm_4-5-1-,OppForm_5-2-3 A,OppForm_5-2-3 B,OppForm_5-3-1-1
0,1,1,FC Groningen,Ajax,Verbruggen,5,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,1,FC Groningen,Ajax,Benedetti,7,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,1,FC Groningen,Ajax,Gabriel Paulista,6,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,1,FC Groningen,Ajax,Thiaw,6,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,1,FC Groningen,Ajax,Dedic,5,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [89]:
# Target encoding the team and player variables using rating
def kfold_target_encode(df, feature_cols, target_col, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    df_encoded = df.copy()
    
    for feature in feature_cols:
        df_encoded[f'{feature}_encoded'] = np.nan
        
    for train_idx, val_idx in kf.split(df):
        train = df.iloc[train_idx]
        val = df.iloc[val_idx]
        
        encoder = TargetEncoder(cols=feature_cols)
        encoder.fit(train[feature_cols], train[target_col])
        
        train_encoded = encoder.transform(train[feature_cols])
        val_encoded = encoder.transform(val[feature_cols])
        
        for feature in feature_cols:
            df_encoded.iloc[train_idx, df_encoded.columns.get_loc(f'{feature}_encoded')] = train_encoded[feature]
            df_encoded.iloc[val_idx, df_encoded.columns.get_loc(f'{feature}_encoded')] = val_encoded[feature]
    
    return df_encoded

# Assuming 'df' is your DataFrame
features_to_encode = ['Team', 'Opponent_x', 'Name']
target = 'Rating'

encoded_df = kfold_target_encode(rating_update, features_to_encode, target)

In [90]:
encoded_df.sample(5)

,Match,Game,Team,Opponent_x,Name,Rating,goal,Assist,yellowcard,redcard,...,OppForm_4-3-3 B,OppForm_4-4-2 A,OppForm_4-4-2 B,OppForm_4-5-1-,OppForm_5-2-3 A,OppForm_5-2-3 B,OppForm_5-3-1-1,Team_encoded,Opponent_x_encoded,Name_encoded
6189,32,5,PEC Zwolle,NAC Breda,Vellios,4,0,0,0,0,...,0,0,1,0,0,0,0,5.539432,6.832215,6.072900
5707,30,3,Sparta Rotterdam,Fortuna Sittard,Mito,6,0,0,0,0,...,0,0,0,0,0,0,0,5.818493,6.758730,6.413195
2136,12,1,Willem II,FC Twente,Sané,9,1,0,0,0,...,1,0,0,0,0,0,0,7.507463,6.688356,6.880497
3186,17,2,RKC Waalwijk,FC Twente,Meijers,3,0,0,0,0,...,0,0,1,0,0,0,0,5.190635,6.688356,5.697900
6400,33,5,NAC Breda,Heracles Almelo,Morgalla,7,0,0,0,0,...,0,0,0,0,0,0,0,6.458065,6.958457,6.725829


In [322]:
# Exporting the encoded dataset
encoded_df.to_csv("D:\Webscraping\Eridivisie\Cleaned_dataset\Deep Learning\encoded.csv", index=False)

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\hoybr\AppData\Local\Temp\ipykernel_26068\2765665668.py:2: SyntaxWarning: invalid escape sequence '\W'
  encoded_df.to_csv("D:\Webscraping\Eridivisie\Cleaned_dataset\Deep Learning\encoded.csv", index=False)
